# Preprocessing, building a Pandas dataframe and saving it as a  .csv file

In [5]:
import re
import sys
import glob
import string
from pprint import pprint
from collections import Counter, OrderedDict

import spacy
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

import pickle

import warnings
warnings.filterwarnings('ignore')

BASE_PATH = '/home/anuja/Documents/anu/EmailRecommmendation/'

folder_path = "/home/anuja/Desktop/mini_deb/*"
file_name = BASE_PATH + "/model/dataframe3.csv"
file_name1 = BASE_PATH + "/model/dataframe4.csv"
file_name2 = BASE_PATH + "/model/dataframe5.csv"
sys.path.insert(0, BASE_PATH + '/Preprocessing')
PATH = BASE_PATH + '/model/first_model.pt'

import preprocessing
import read_file
import datetime

def extract_debian(text):
    text = text.split('\n\n\n')
    header = text[0].split('\n')
    body = text[1]
    sender = header[2].split(':')[1].split('<')[0]
#     print('Sender',sender)
#     print('Body \n',body)
    return sender,body

def clean_debian(temp):
    temp = re.sub('\n+','\n',temp)
    temp = re.sub('\n',' ',temp)
    temp = re.sub('\t',' ',temp)
    temp = re.sub(' +',' ',temp)
    return temp

def deb_lemmatize(doc):        
    doc = nlp(doc)
    article, skl_texts = '',''
    for w in doc:
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            article += " " + w.lemma_
        if w.text == '\n':                
            skl_texts += " " + article
            article = ''       
    return skl_texts

def deb_toppostremoval(temp):
    strings = temp.splitlines()
    temp = ''
    for st in strings:
        st = st.strip()
        if len(st)>0:
            if st[0]=='>':
                continue
            else:
                temp += '\n' + st
    return temp

df = pd.DataFrame()
folder = glob.glob(folder_path)
obj = preprocessing.preprocess()
count_file = 0
thread_list=[]
try:
    for fol in tqdm_notebook(folder):
        files = glob.glob(fol+'/*.txt')
        threads = []
        for file in files:
            ob = read_file.file_content(file)
            ob.read_file_content()
            threads.append(ob.mail)
            count_file += 1
        sorted_threads = sorted(threads, key=lambda ke: datetime.datetime.strptime(ke['Date'],'%a, %d %b %Y %H:%M:%S %z'))
        thread_list.append(sorted_threads)
except:
    print(fol)
print(len(thread_list))


23


In [7]:
from random import randint
import numpy as np
import torch
from models import InferSent
model_version = 1
MODEL_PATH = "/home/anuja/Desktop/BE project/Models/InferSent/infersent1.pkl"
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
infermodel = InferSent(params_model)
infermodel.load_state_dict(torch.load(MODEL_PATH))
use_cuda = False
infermodel = infermodel.cuda() if use_cuda else infermodel
W2V_PATH = '/home/anuja/Desktop/BE project/glove.6B/glove.840B.300d.txt'
#replace with glove.840B.300d.txt
infermodel.set_w2v_path(W2V_PATH)
infermodel.build_vocab_k_words(K=100000)

Vocab size : 100000


In [8]:
df_trn = pd.DataFrame()
df_tst = pd.DataFrame()
split_date = datetime.datetime.strptime('01 Sep 2017 23:01:14 +0000', '%d %b %Y %H:%M:%S %z')

users = []
trn_users = []
tst_users = []
th_no = 0
cnt = 0
for thr in thread_list:
    start_date = ""
    flag = 0
    t = ''
    for mail in thr:
        temp = ''
        sender = mail['From'].split('<')[0].strip()
        temp   = mail['content']
        users.append(sender)
        temp = deb_toppostremoval(temp)
        temp = deb_lemmatize(temp)
        temp = clean_debian(temp)
        if temp == '':
            cnt += 1
            continue
        temp = obj.replace_tokens(temp)
        if temp!='' :
            embedding =infermodel.encode( str(temp), bsize=1, tokenize=False, verbose=True)
            sent_vec =[]
            numw = 0
            for w in embedding:
                try:
                    if numw == 0:
                        sent_vec = w
                    else:
                        sent_vec = np.add(sent_vec, w)
                    numw+=1
                except:
                    pass
            v = np.asarray(sent_vec) / numw
#             print(v.shape)
#             print(v)
            v=np.transpose(v)
#             print(v.shape)
            print(v)
        if flag==0:
            start_date = datetime.datetime.strptime(mail['Date'],'%a, %d %b %Y %H:%M:%S %z')
            if start_date > split_date:
                df_tst = df_tst.append({'body': str(temp),'replier':sender, 'thread_no':th_no, 'start_date':start_date,'embeddings':v}, ignore_index=True)
                tst_users.append(sender)
            else:
                trn_users.append(sender)
            t = temp
            flag = 1
            continue

        
        df = df.append({'body': str(t),'replier':sender, 'thread_no':th_no, 'start_date':start_date,'embeddings':v}, ignore_index=True)

        if start_date <= split_date:
            df_trn = df_trn.append({'body': str(t),'replier':sender, 'thread_no':th_no, 'start_date':start_date,'embeddings':v}, ignore_index=True)
            trn_users.append(sender)
            t = t + temp
        else:
            df_tst = df_tst.append({'body': str(temp),'replier':sender, 'thread_no':th_no, 'start_date':start_date,'embeddings':v}, ignore_index=True)
            tst_users.append(sender)

        
        #t = t + temp
    th_no += 1

print(cnt)
print(count_file)
print(len(df['body']))
print(len(df['thread_no'].unique()))
print(len(df['replier'].unique()))
print(len(df['embeddings'][0]))
rep_to_index = {}
index = 0
for rep in users:
    if rep_to_index.get(rep, 0) == 0:
        rep_to_index[rep] = index
        index += 1
pprint(rep_to_index)


for rep in df_trn['replier']:
    df_trn.loc[df_trn['replier']==rep,'int_replier'] = rep_to_index[rep]
#print(df_trn.head)    

for rep in df_tst['replier']:
    df_tst.loc[df_tst['replier']==rep,'int_replier'] = rep_to_index[rep]

for rep in df['replier']:
    df.loc[df['replier']==rep,'int_replier'] = rep_to_index[rep]
    
for rep in df['replier']:
    df.loc[df['replier']==rep,'int_replier'] = rep_to_index[rep]
    

#Offset the replier in test dataframe

df_tst['replier'] = df_tst.groupby('thread_no')['replier'].shift(-1)
df_tst['int_replier'] = df_tst.groupby('thread_no')['int_replier'].shift(-1)

df_tst.dropna(inplace=True)

df_trn.to_csv(file_name)
df_tst.to_csv(file_name1)
df.to_csv(file_name2)

Nb words kept : 2087/2087 (100.0%)
Speed : 20.2 sentences/s (cpu mode, bsize=1)
[ 0.03650786 -0.04985265 -0.02311382 ... -0.02699613 -0.02101633
  0.00768024]
Nb words kept : 61/61 (100.0%)
Speed : 22.9 sentences/s (cpu mode, bsize=1)
[ 0.0358405  -0.0489863  -0.02340721 ... -0.03197146 -0.0195006
  0.00036863]
Nb words kept : 1274/1274 (100.0%)
Speed : 23.0 sentences/s (cpu mode, bsize=1)
[ 0.03571672 -0.04736275 -0.02346319 ... -0.02552481 -0.02052212
  0.00722022]
Nb words kept : 426/426 (100.0%)
Speed : 23.3 sentences/s (cpu mode, bsize=1)
[ 0.03380246 -0.04779268 -0.02394018 ... -0.02711713 -0.02011451
  0.00934879]
Nb words kept : 185/185 (100.0%)
Speed : 23.3 sentences/s (cpu mode, bsize=1)
[ 0.03249109 -0.04508093 -0.02424568 ... -0.03202257 -0.01960267
  0.00985559]
Nb words kept : 1060/1060 (100.0%)
Speed : 23.6 sentences/s (cpu mode, bsize=1)
[ 0.03561683 -0.04890719 -0.02283118 ... -0.02679757 -0.02073223
  0.00711691]
Nb words kept : 926/926 (100.0%)
Speed : 19.1 sentences

Speed : 20.9 sentences/s (cpu mode, bsize=1)
[ 0.03568519 -0.04481908 -0.02233277 ... -0.02482249 -0.02047235
  0.00748657]
Nb words kept : 342/342 (100.0%)
Speed : 20.5 sentences/s (cpu mode, bsize=1)
[ 0.0319744  -0.04911947 -0.0237011  ... -0.02877466 -0.02015302
  0.00337766]
Nb words kept : 206/206 (100.0%)
Speed : 20.8 sentences/s (cpu mode, bsize=1)
[ 0.03349618 -0.04706893 -0.02318899 ... -0.02599528 -0.02113654
  0.01188441]
Nb words kept : 171/171 (100.0%)
Speed : 21.3 sentences/s (cpu mode, bsize=1)
[ 0.03582375 -0.05241599 -0.02550579 ... -0.03193107 -0.02023388
  0.00749555]
Nb words kept : 135/135 (100.0%)
Speed : 21.0 sentences/s (cpu mode, bsize=1)
[ 0.03446902 -0.05008711 -0.02461352 ... -0.0275008  -0.02051609
  0.01554701]
Nb words kept : 680/680 (100.0%)
Speed : 21.0 sentences/s (cpu mode, bsize=1)
[ 0.03550123 -0.05193973 -0.02371666 ... -0.02561482 -0.02042306
  0.00408496]
Nb words kept : 1117/1117 (100.0%)
Speed : 21.0 sentences/s (cpu mode, bsize=1)
[ 0.0340815

## Indexing of words in vocab

In [ ]:
words = Counter()
for sent in df_trn.body.values:
    words.update(w.text.lower() for w in nlp(sent))
# print(words)

words = sorted(words, key=words.get, reverse=True)
print(words)
words = ['_PAD','_UNK'] + words

word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

def indexer(s):
    vec = []
    for wr in nlp(s):
        wr = wr.text.lower()
        if wr in word2idx:
            vec.append(word2idx[wr])
        else:
            vec.append(word2idx['_PAD'])
    return vec

# User Vector - construction

In [2]:
np.set_printoptions(threshold = sys.maxsize)
user_indices = []
trn_user_indices = []
tst_user_indices = []

for u in users:
    user_indices.append(rep_to_index[u])

for v in trn_users:
    trn_user_indices.append(rep_to_index[v])

for w in tst_users:
    tst_user_indices.append(rep_to_index[w])

NameError: name 'users' is not defined

In [3]:
user_vec_len = max(user_indices) + 1

ValueError: max() arg is an empty sequence

In [ ]:
indexx=0
weight_list = []
for i in range(0, df_trn.thread_no.shape[0]+1):
    temp_index=indexx
    array  = np.zeros(user_vec_len)
    for j in range(temp_index, temp_index + list(df_trn.thread_no).count(i)):
        array[trn_user_indices[j]] += 1
        weight_list.append(list(array))
        indexx+=1

trn_weights = np.array(weight_list)


In [ ]:
indexx=0
weight_list = []
for i in range(0, df_tst.thread_no.shape[0]+1):
    temp_index=indexx
    array  = np.zeros(user_vec_len)
    for j in range(temp_index, temp_index + list(df_tst.thread_no).count(i)):
        array[tst_user_indices[j]] += 1
        weight_list.append(list(array))
        indexx+=1

tst_weights = np.array(weight_list)

# Dataset Loading

In [ ]:
# embedding |> flag
class VectorizeData(Dataset):
    def __init__(self, df_path, maxlen=10, calc_maxlen = False):
        self.df = pd.read_csv(df_path, error_bad_lines=False)
        self.df['body'] = self.df.body.apply(lambda x: x.strip())
        print('Indexing...')
        self.df['bodyidx'] = self.df.body.apply(indexer)
        print('Calculating lengths')
        self.df['lengths'] = self.df.bodyidx.apply(len)
        if calc_maxlen == True:
            self.maxlen = max(self.df['lengths'])
        else:
            self.maxlen = maxlen
        print(self.maxlen)
        print('Padding')
        self.df['bodypadded'] = self.df.bodyidx.apply(self.pad_data)
        print(self.df)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        X = self.df.bodypadded[idx]
        lens = self.df.lengths[idx]
        y = self.df.int_replier[idx]
        e=self.df.embeddings[idx]
        return X,y,lens,e
    
    def pad_data(self, s):
        padded = np.zeros((self.maxlen,), dtype=np.int64)
        if len(s) > self.maxlen: padded[:] = s[:self.maxlen]
        else: padded[:len(s)] = s
        return padded

In [ ]:
ds = VectorizeData(file_name2, calc_maxlen = True)
dtrain = VectorizeData(file_name, maxlen = ds.maxlen)
dtest = VectorizeData(file_name1, maxlen = ds.maxlen)

# Pytorch Feedforward Neural Network model

In [ ]:
input_size = ds.maxlen
hidden_size = 50
num_classes = user_vec_len
num_epochs = 5
batch_size = 1
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# embeddings |>
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size,user_vec_len, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size + user_vec_len, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x,w):
#         x = torch.FloatTensor(x) 
        catt = torch.cat((x,w),1)
        out = self.fc1(catt)
        out = self.relu(out)       
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out


In [ ]:
model = NeuralNet(input_size, hidden_size,user_vec_len, num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
train_dl= DataLoader(dtrain, batch_size=1)
num_batch = len(train_dl)
for epoch in range(num_epochs):
    y_true_train = list()
    y_pred_train = list()
    total_loss_train = 0
    t = tqdm_notebook(iter(train_dl), leave=False, total=num_batch)
    for we, w in zip(t,trn_weights):
        X = we[0]
        y = we[1]
        lengths = we[2]
        
        w = w.reshape(-1,1)
        w = w.transpose()
        
        w = Variable(torch.Tensor(w).cpu())
        X = Variable(X.cpu())
        y = Variable(y.cpu())
        lengths = lengths.numpy()

        opt.zero_grad()
        X = X.float()
        w = w.float()
        y = y.long()
        pred = model(X,w)
        # F.nll_loss can be replaced with criterion
        loss = F.nll_loss(pred, y)
        loss.backward()
        opt.step()

        t.set_postfix(loss=loss.item())
        pred_idx = torch.max(pred, dim=1)[1]

        y_true_train += list(y.cpu().data.numpy())
        y_pred_train += list(pred_idx.cpu().data.numpy())
        total_loss_train += loss.item()
        

    train_acc = accuracy_score(y_true_train, y_pred_train)
    train_loss = total_loss_train/len(train_dl)
    print(f' Epoch {epoch}: Train loss: {train_loss} acc: {train_acc}')
torch.save(model.state_dict(),PATH)

# architecture 
# loading pickle file and predict


# Testing 

In [ ]:
test_dl= DataLoader(dtest, batch_size=1)
num_batches = len(test_dl)
y_true_test1 = list()
y_pred_test1 = list()
total_loss_train = 0
tt = tqdm_notebook(iter(test_dl), leave=False, total=num_batch)
for we, w in zip(tt,tst_weights):
    X = we[0]
    y = we[1]
    lengths = we[2]
    
    w = w.reshape(-1,1)
    w = w.transpose()

    w = Variable(torch.Tensor(w).cpu())
    X = Variable(X.cpu())
    y = Variable(y.cpu())
    lengths = lengths.numpy()

    X = X.float()
    w = w.float()
    y = y.long()
    pred = model(X,w)
    loss = F.nll_loss(pred, y)


    y_true_train += list(y.cpu().data.numpy())
    y_pred_train += list(pred_idx.cpu().data.numpy())
    total_loss_train += loss.item()

train_acc = accuracy_score(y_true_train, y_pred_train)
train_loss = total_loss_train/len(train_dl)
print(f' Epoch {epoch}: Train loss: {train_loss} acc: {train_acc}')